In [4]:
# Step 1: Data Collection and Preparation
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

print("Libraries imported successfully.")

# Paths to the dataset directories
train_dir = r'C:\Users\danie\sign-language-alpha\data\asl_alphabet_train\asl_alphabet_train'
test_dir = r'C:\Users\danie\sign-language-alpha\data\asl_alphabet_test\asl_alphabet_test'

# Step 1: Preprocessing Images
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (64, 64))
    return image

print("Preprocessing function defined.")

# Step 2: Use MediaPipe for Hand Landmarks Extraction
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def extract_hand_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks
        keypoints = []
        for handLms in hand_landmarks:
            for lm in handLms.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
        return np.array(keypoints).flatten()
    return np.zeros(21 * 3 * 2)  # Return zero array if no hands detected

print("MediaPipe setup completed.")

# Step 3: Create a DataFrame to Store Landmark Information
columns = [f'x{i}' for i in range(21*2)] + [f'y{i}' for i in range(21*2)] + [f'z{i}' for i in range(21*2)] + ['label']
df = pd.DataFrame(columns=columns)

print("DataFrame initialized.")

# Populate DataFrame
rows = []
for label in os.listdir(train_dir):
    label_dir = os.path.join(train_dir, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        image = preprocess_image(img_path)
        keypoints = extract_hand_keypoints(image)
        row = np.append(keypoints, label)
        rows.append(row)

df = pd.concat([df, pd.DataFrame(rows, columns=columns)], ignore_index=True)

print("DataFrame populated with landmark data.")

# Step 4: Model Training
# Define Model Architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(21*3*2,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(os.listdir(train_dir)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Model architecture defined.")

# Prepare Training Data
X = df.drop('label', axis=1).values
y = df['label'].factorize()[0]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training and validation data prepared.")

# Train the Model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

print("Model training completed.")

# Save the model and label encoder
model.save('sign_language_model.h5')
np.save('label_encoder_classes.npy', df['label'].factorize()[1])

print("Model and label encoder saved.")

# Step 5: Live Translation Using OpenCV
# Setup OpenCV for Real-Time Capture
cap = cv2.VideoCapture(0)

print("Starting real-time capture. Press 'q' to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    keypoints = extract_hand_keypoints(frame)
    keypoints = keypoints.reshape(1, -1)  # Reshape for the model input
    
    predictions = model.predict(keypoints)
    predicted_label = df['label'].factorize()[1][np.argmax(predictions)]
    print(f'Predicted Label: {predicted_label}')

    # Display the frame
    cv2.putText(frame, f'Predicted: {predicted_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Real-time capture ended.")

# Step 6: Testing and Refinement
# (This part is generally iterative and requires manual adjustments and testing in real-world scenarios)

print("Testing and refinement phase. Evaluate the model's performance and make necessary adjustments.")


Libraries imported successfully.
Preprocessing function defined.
MediaPipe setup completed.
DataFrame initialized.


C:\Users\danie\anaconda3\envs\myenv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
